In [343]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiLineString, LineString
from shapely.ops import nearest_points
import math
import numpy as np

## Load in Data

In [344]:
data_dir = "Bus_RoutesStopsServices"
bus_routes = pd.read_csv(f"{data_dir}/bus_routes.csv")
bus_services = pd.read_csv(f"{data_dir}/bus_services.csv")
bus_stops = pd.read_csv(f"{data_dir}/bus_stops.csv")

# bus services data: filter for trunk, and direction = 1
trunk = bus_services[bus_services['Category'] == 'TRUNK']
trunk = trunk[trunk['Direction'] == 1]

#bus routes data: filter for direction = 1 and trunk buses
bus_routes = bus_routes[bus_routes['Direction'] == 1]
merged_data = pd.merge(bus_routes, trunk[['ServiceNo']], on='ServiceNo', how='inner')
merged_data

# join to get the bus stop coordinates
final_data = pd.merge(merged_data, bus_stops, on='BusStopCode', how='left')

In [345]:
train_stations = gpd.read_file("TrainStation_Jul2024/repaired_shapefile.shp")

mrt_lines = {
    'Thomson-East Coast Line': ['WOODLANDS NORTH MRT STATION','WOODLANDS MRT STATION','WOODLANDS SOUTH MRT STATION', 'SPRINGLEAF MRT STATION','LENTOR MRT STATION','MAYFLOWER MRT STATION','BRIGHT HILL MRT STATION',
                                'UPPER THOMSON MRT STATION','CALDECOTT MRT STATION','STEVENS MRT STATION','NAPIER MRT STATION','ORCHARD BOULEVARD MRT STATION','ORCHARD MRT STATION','GREAT WORLD MRT STATION',
                                'HAVELOCK MRT STATION','OUTRAM PARK MRT STATION','MAXWELL MRT STATION','SHENTON WAY MRT STATION','MARINA BAY MRT STATION','GARDENS BY THE BAY MRT STATION','TANJONG RHU MRT STATION',
                                'KATONG PARK MRT STATION','TANJONG KATONG MRT STATION','MARINE PARADE MRT STATION','MARINE TERRACE MRT STATION','SIGLAP MRT STATION','BAYSHORE MRT STATION'],
    'North-South Line': ['JURONG EAST MRT STATION','BUKIT BATOK MRT STATION','BUKIT GOMBAK MRT STATION','CHOA CHU KANG MRT STATION','YEW TEE MRT STATION','KRANJI MRT STATION','MARSILING MRT STATION','WOODLANDS MRT STATION',
                         'ADMIRALTY MRT STATION','SEMBAWANG MRT STATION','CANBERRRA MRT STATION','YISHUN MRT STATION','KHATIB MRT STATION','YIO CHU KANG MRT STATION','ANG MO KIO MRT STATION','BISHAN MRT STATION',
                         'BRADDELL MRT STATION','TOA PAYOH MRT STATION','NOVENA MRT STATION','NEWTON MRT STATION','ORCHARD MRT STATION','SOMERSET MRT STATION','DHOBY GHAUT MRT STATION','CITY HALL MRT STATION',
                         'RAFFLES PLACE MRT STATION','MARINA BAY MRT STATION','MARINA SOUTH PIER MRT STATION'],
    'Circle Line': ['DHOBY GHAUT MRT STATION', 'BRAS BASAH MRT STATION', 'ESPLANADE MRT STATION', 'PROMENADE MRT STATION','NICOLL HIGHWAY MRT STATION','STADIUM MRT STATION',
                    'MOUNTBATTEN MRT STATION','DAKOTA MRT STATION','PAYA LEBAR MRT STATION','MACPHERSON MRT STATION','TAI SENG MRT STATION','BARTLEY MRT STATION','SERANGOON MRT STATION','LORONG CHUAN MRT STATION',
                    'BISHAN MRT STATION','MARYMOUNT MRT STATION','CALDECOTT MRT STATION','BOTANIC GARDENS MRT STATION','FARRER ROAD MRT STATION','HOLLAND VILLAGE MRT STATION','BUONA VISTA MRT STATION','ONE-NORTH MRT STATION',
                    'KENT RIDGE MRT STATION', 'HAW PAW VILLA MRT STATION', 'PASIR PANJANG MRT STATION', 'LABRADOR PARK MRT STATION', 'TELOK BLANGAH MRT STATION','HABOURFRONT MRT STATION'],
    'Circle Line.': ['PROMENADE MRT STATION','BAYFRONT MRT STATION', 'MARINA BAY MRT STATION'],
    'North-East Line': ['HARBOURFRONT MRT STATION','OUTRAM PARK MRT STATION','CHINATOWN MRT STATION','CLARKE QUAY MRT STATION','DHOBY GHAUT MRT STATION','LITTLE INDIA MRT STATION','FARRER PARK MRT STATION','BOON KENG MRT STATION',
                        'POTONG PASIR MRT STATION','WOODLEIGH MRT STATION','SERANGOON MRT STATION','KOVAN MRT STATION','HOUGANG MRT STATION','BUANGKOK MRT STATION','SENGKANG MRT STATION','PUNGGOL MRT STATION'],
    'Downtown Line': ['BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 'SIXTH AVENUE','TAN KAH KEE MRT STATION',
                      'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION',
                      'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', ' BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 'GEYLANG  BAHRU MRT STATION',
                      'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', ' BEDOK NORTH MRT STATION', 'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION',
                      'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'],
    'East-West Line': ['PASIR RIS MRT STATION', 'TAMPINES MRT STATION', 'SIMEI MRT STATION', 'TANAH MERAH MRT STATION','BEDOK MRT STATION', 'KEMBANGAN MRT STATION', 'EUNOS MRT STATION', 'PAYA LEBAR MRT STATION',
                       'ALJUNIED MRT STATION', 'KALLANG MRT STATION', ' LAVENDER MRT STATION','BUGIS MRT STATION', 'CITY HALL MRT STATION', 'RAFFLES PLACE MRT STATION','TANJONG PAGAR MRT STATION', 'OUTRAM PARK MRT STATION',
                       'TIONG BAHRU MRT STATION', 'REDHILL MRT STATION', 'QUEENSTOWN MRT STATION', 'COMMONWEALTH MRT STATION','BUONA VISTA MRT STATION', 'DOVER MRT STATION', 'CLEMENTI MRT STATION', 'JURONG EAST MRT STATION',
                       'CHINESE GARDEN MRT STATION', 'LAKESIDE MRT STATION', 'BOON LAY MRT STATION', 'PIONEER MRT STATION','JOO KON MRT STATION',' GUL CIRCLE MRT STATION', 'TUAS CRESCENT MRT STATION', 
                       'TUAS WEST ROAD MRT STATION', 'TUAS LINK MRT STATION'],
    'East-West Line.':['TANAH MERAH MRT STATION','EXPO MRT STATION', 'CHANGI AIRPORT MRT STATION']
}
line_colors = {
    'North-South Line': 'red',
    'East-West Line': 'green',
    'East-West Line.': 'green',
    'Circle Line': 'darkorange',
    'Circle Line.': 'darkorange',
    #'Circle Line.': 'black', # can use this because orange hard to see
    #'Circle Line': 'black',
    'North-East Line': 'purple',
    'Downtown Line': 'blue',
    'Thomson-East Coast Line': 'brown'
}

mrt_line_data = []
for line, stations in mrt_lines.items():
    for index, station in enumerate(stations):
        mrt_line_data.append((station, line, index+1))
mrt_line_df = pd.DataFrame(mrt_line_data, columns=['STN_NAM_DE', 'MRT_LINE', 'STN_SEQUENCE'])

# Merge this DataFrame with your original train_stations DataFrame on the STN_NAM_DE column
train_stations_merged = train_stations.merge(mrt_line_df, on='STN_NAM_DE', how='left')
merged_stations_unique = train_stations_merged.drop_duplicates(subset=['STN_NAM_DE', 'MRT_LINE'])
merged_stations_unique = merged_stations_unique.to_crs(epsg=4326)
grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])
grouped_train_lines = grouped_train_lines.sort_values(['MRT_LINE','STN_SEQUENCE'])
grouped_train_lines = grouped_train_lines.to_crs(epsg=4326)


/Users/krystal/.virtualenvs/r-tensorflow/lib/python3.10/site-packages/pyogrio/raw.py:196: RuntimeWarning: TrainStation_Jul2024/repaired_shapefile.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/1632610050.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_train_lines = merged_stations_unique.groupby(['MRT_LINE']).apply(lambda x: x[['STN_NAM_DE', 'STN_SEQUENCE','geometry']])


## Calculative Functions

In [346]:
def haversine(coord1, coord2): ## ccomputes distance between two points on the surface of a sphere using the latitude and longitude  >> supposed to be better than eucidian 
    R = 6371.0  # Radius of Earth in kilometers
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

def calculate_bearing(coord1, coord2):
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    delta_lon = lon2 - lon1
    x = np.sin(delta_lon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon))
    bearing = np.degrees(np.arctan2(x, y))
    return (bearing + 360) % 360

In [347]:
def calculate_route_parallelism(bus_route_line, mrt_multiline):
    total_score = 0
    segment_scores = []
    total_length = 0  # Total length of the bus route in meters

    # Loop through each segment in the bus route
    for i in range(len(bus_route_line.coords) - 1):
        # Define the start and end points of the bus segment
        bus_segment = LineString([bus_route_line.coords[i], bus_route_line.coords[i + 1]])
        bus_bearing = calculate_bearing(bus_route_line.coords[i], bus_route_line.coords[i + 1])
        
        # Calculate the length of the bus segment (in meters)
        segment_length = haversine(bus_route_line.coords[i], bus_route_line.coords[i + 1]) * 1000
        total_length += segment_length

        # Find the nearest MRT line segment to this bus segment
        min_distance = float('inf')
        nearest_mrt_bearing = None

        # Loop through each MRT line segment in the MRT network
        for mrt_line in mrt_multiline.geoms:
            for j in range(len(mrt_line.coords) - 1):
                mrt_segment = LineString([mrt_line.coords[j], mrt_line.coords[j + 1]])
                mrt_bearing = calculate_bearing(mrt_line.coords[j], mrt_line.coords[j + 1])

                # Find the minimum distance from the bus segment to the MRT segment
                p1, p2 = nearest_points(bus_segment, mrt_segment)
                distance = haversine((p1.y, p1.x), (p2.y, p2.x)) * 1000

                if distance < min_distance:
                    min_distance = distance
                    nearest_mrt_bearing = mrt_bearing

        # Calculate the bearing difference between the bus segment and nearest MRT segment
        bearing_diff = abs(bus_bearing - nearest_mrt_bearing)
        if bearing_diff > 180:
            bearing_diff = 360 - bearing_diff

        # Calculate the parallelism score for this segment, considering both distance and bearing difference
        score = (1 / (1 + min_distance)) * (1 - bearing_diff / 180)  # Score decreases as distance or bearing diff increases

        # Weight the score by the segment length but smooth its influence by scaling
        # Using sqrt(segment_length) to reduce the extreme influence of very long segments without a hard cap
        weighted_score = score * np.sqrt(segment_length)
        segment_scores.append(weighted_score)
        total_score += weighted_score

    # Normalize the total score by the square root of the total route length
    # Smooths out the impact of route length and avoids inflating scores for shorter routes
    normalized_score = total_score / np.sqrt(total_length) if total_length > 0 else 0

    return normalized_score

#def calculate_route_parallelism(bus_route_line, mrt_multiline): ## WITHOUT SMOOTHING FOR WEIGHTAGE
    total_score = 0
    segment_scores = []
    total_length = 0  # Total length of the bus route
    
    # Loop through each segment in the bus route
    for i in range(len(bus_route_line.coords) - 1):
        # Define the start and end points of the bus segment
        bus_segment = LineString([bus_route_line.coords[i], bus_route_line.coords[i + 1]])
        bus_bearing = calculate_bearing(bus_route_line.coords[i], bus_route_line.coords[i + 1])
        
        # Calculate the length of the bus segment (in meters)
        segment_length = haversine(bus_route_line.coords[i], bus_route_line.coords[i + 1]) * 1000
        total_length += segment_length

        # Find the nearest MRT line segment to this bus segment
        min_distance = float('inf')
        nearest_mrt_bearing = None

        # Loop through each MRT line segment in the MRT network
        for mrt_line in mrt_multiline.geoms:
            for j in range(len(mrt_line.coords) - 1):
                mrt_segment = LineString([mrt_line.coords[j], mrt_line.coords[j + 1]])
                mrt_bearing = calculate_bearing(mrt_line.coords[j], mrt_line.coords[j + 1])

                # Find the minimum distance from the bus segment to the MRT segment
                p1, p2 = nearest_points(bus_segment, mrt_segment)
                distance = haversine((p1.y, p1.x), (p2.y, p2.x)) * 1000

                if distance < min_distance:
                    min_distance = distance
                    nearest_mrt_bearing = mrt_bearing

        # Calculate the bearing difference between the bus segment and nearest MRT segment
        bearing_diff = abs(bus_bearing - nearest_mrt_bearing)
        if bearing_diff > 180:
            bearing_diff = 360 - bearing_diff

        # Calculate the parallelism score for this segment
        score = (1 / (1 + min_distance)) * (1 - bearing_diff / 180)  # Score decreases as distance and bearing diff increases
        
        # Weight the score by the segment length
        weighted_score = score * segment_length
        segment_scores.append(weighted_score)
        total_score += weighted_score

    # Normalize the total score by the total length of the bus route
    average_score = total_score / total_length if total_length > 0 else 0
    
    return average_score

#def calculate_route_parallelism(bus_route_line, mrt_multiline): ## WITHOUT CONSIDERING LENGTH OF SEGMENT 
    total_score = 0
    segment_scores = []
    total_segments = len(bus_route_line.coords) - 1  # Number of segments in the bus route

    # Loop through each segment in the bus route
    for i in range(total_segments):
        # Define the start and end points of the bus segment
        bus_segment = LineString([bus_route_line.coords[i], bus_route_line.coords[i + 1]])
        bus_bearing = calculate_bearing(bus_route_line.coords[i], bus_route_line.coords[i + 1])
        
        # Find the nearest MRT line segment to this bus segment
        min_distance = float('inf')
        nearest_mrt_segment = None
        nearest_mrt_bearing = None

        # Loop through each MRT line segment in the MRT network
        for mrt_line in mrt_multiline.geoms:
            for j in range(len(mrt_line.coords) - 1):
                mrt_segment = LineString([mrt_line.coords[j], mrt_line.coords[j + 1]])
                mrt_bearing = calculate_bearing(mrt_line.coords[j], mrt_line.coords[j + 1])

                # Find the minimum distance from the bus segment to the MRT segment
                p1, p2 = nearest_points(bus_segment, mrt_segment)
                distance = haversine((p1.y, p1.x), (p2.y, p2.x)) * 1000

                # If this is the closest MRT segment so far, store it
                if distance < min_distance:
                    min_distance = distance
                    nearest_mrt_segment = mrt_segment
                    nearest_mrt_bearing = mrt_bearing

        # Calculate the bearing difference between the bus segment and nearest MRT segment > assess how parallel
        bearing_diff = abs(bus_bearing - nearest_mrt_bearing)
        if bearing_diff > 180:
            bearing_diff = 360 - bearing_diff

        # Calculate the parallelism score for this segment:
        # Considers both distance and bearing difference.
        # Score decreases as distance or bearing difference increases.
        score = 1 / (1 + min_distance) * (1 - bearing_diff / 180)  # Normalize bearing_diff to 0-1 range

        segment_scores.append(score)
        total_score += score

    # The overall parallelism score is the average of all segment scores
    average_score = total_score / total_segments if total_segments > 0 else 0
    return average_score


## Generate Parallelism Score

In [348]:
# Treat MRT Lines as One MultiString 
mrt_lines_list = []
grouped_train_lines_multi = grouped_train_lines.groupby(level='MRT_LINE') 

for MRT_LINE, group in grouped_train_lines_multi:
    group_sorted = group.sort_values('STN_SEQUENCE')
    coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
    mrt_lines_list.append(LineString(coordinates))

# Merge into one MultiLineString
mrt_multiline = MultiLineString(mrt_lines_list)


parallelism_results = []

unique_services = final_data['ServiceNo'].unique()

for service_no in unique_services:
    busroutes = final_data[final_data['ServiceNo'] == service_no]
    grouped_bus_routes = busroutes.groupby(['ServiceNo', 'Direction'])
    
    for (service_no, direction), group in grouped_bus_routes:
        group_sorted = group.sort_values('StopSequence')  # Ensure bus stops are in correct sequence
        bus_coordinates = list(zip(group_sorted['Latitude'], group_sorted['Longitude']))  # Get coordinates
        bus_route_line = LineString(bus_coordinates)  # Plot the line
        
        # Calculate parallelism score for the bus route against the entire MRT network
        parallel_score = calculate_route_parallelism(bus_route_line, mrt_multiline)
        
        # Store the result in the parallelism_results list as a dictionary
        parallelism_results.append({
            'ServiceNo': service_no,
            'Direction': direction,
            'ParallelismScore': parallel_score
        })


parallelism_df = pd.DataFrame(parallelism_results)
parallelism_df = parallelism_df.sort_values(by='ParallelismScore', ascending=False)

# Ranked 
print(parallelism_df)


/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/3926433673.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/3926433673.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/3926433673.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord

    ServiceNo  Direction  ParallelismScore
249        63          1          2.254552
352      961M          1          1.964276
73        145          1          1.927232
197        36          1          1.901650
250       63A          1          1.872716
..        ...        ...               ...
242         6          1          0.004168
45       127A          1          0.003957
373      973A          1          0.003083
257       68A          1          0.003008
330       88B          1          0.002643

[402 rows x 3 columns]


## Visualise for Individual Services

In [349]:
busroutes = final_data[final_data['ServiceNo'].isin(['63'])]
grouped_bus_routes = busroutes.groupby(['ServiceNo', 'Direction'])

singapore = folium.Map(location=(1.359394, 103.814301), zoom_start=12)

for (service_no, direction), group in grouped_bus_routes:
    group_sorted = group.sort_values('StopSequence')  # Ensure bus stops are in correct sequence
    bus_coordinates = list(zip(group_sorted['Latitude'], group_sorted['Longitude']))  # Get coordinates
    bus_route_line = LineString(bus_coordinates)  # Plot the line
    
    # Plot bus routes on the map
    folium.PolyLine(
        locations=bus_coordinates,  
        weight=2,          
        color='black',     
        opacity=0.7,      
        popup=f"Service {service_no}"  # Label with bus service number
    ).add_to(singapore)

# Add bus stop points to the map
for index, row in busroutes.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Bus Stop: {row['BusStopCode']} (Service {row['ServiceNo']})"
    ).add_to(singapore)

# Plot MRT lines and stations
grouped_train_lines = grouped_train_lines.groupby(level='MRT_LINE') 
for MRT_LINE, group in grouped_train_lines:
    group_sorted = group.sort_values('STN_SEQUENCE')
    train_coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
    train_route_line = LineString(train_coordinates)  # Create LineString for MRT line
    line_color = line_colors.get(MRT_LINE, 'black')  # Get line color

    # Plot individual MRT lines on the map
    folium.PolyLine(     
        locations=train_coordinates,  # Use individual MRT line coordinates
        weight=2,          
        color=line_color,     
        opacity=0.7,      
        popup=f"{MRT_LINE}"  # Label with MRT line name
    ).add_to(singapore)

# Add bus stop points to the map
for index, row in busroutes.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Bus Stop: {row['BusStopCode']} (Service {row['ServiceNo']})"
    ).add_to(singapore)
    
    # Plot bus routes in the map
    folium.PolyLine(
        locations=coordinates,  
        weight=2,          
        color='linecolour',     
        opacity=0.7,      
        popup=f"Service {service_no}"  # label with bus service number
    ).add_to(singapore)

    # Calculate parallelism score for the bus route against the entire MRT network
parallel_score = calculate_route_parallelism(bus_route_line, mrt_multiline)
print(f"Parallelism score for Bus {service_no}: {parallel_score}")

# Display the map
singapore


/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/650950102.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/650950102.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_coordinates = group_sorted['geometry'].centroid.apply(lambda geom: (geom.y, geom.x)).tolist()
/var/folders/04/m1t5w2x550x1wh8c434jplpw0000gn/T/ipykernel_54470/650950102.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operati

Parallelism score for Bus 63: 2.2545517825752395
